In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, date
import requests
import time
import pickle
import pandas_datareader as web

In [2]:
# Access stock symbols from csv
symbols = pd.read_csv("Symbol_DAX.csv", delimiter=";")
symbols = symbols["Symbol"]

In [3]:
URL_AlphaVantage = "https://www.alphavantage.co/query?"

In [4]:
today = date.today()
one_year_ago = today.replace(year=today.year-1)

In [7]:
#API Call with multiple stock symbols
#symbols= ["ADDYY","AIR","ALV","BAS","BAYN", "BMW","BNR","CON","1COV","DTG","DHER","DBK","DB1","DPW","DTE","EOAN","FRE","FME","HNR1","HEI","HFG","HEN3","IFX","LIN","MBG","MRK","MTX","MUV2","PAH3","PUM","QIA","RWE","SAP","SRT3","SIE","SHL","SY1","VOW3","VNA","ZAL"]
av_key = "C8QYMTKO2IMFJD27"
df_dict = {}
for symbol in symbols:
    stock = web.DataReader(symbol, 'av-daily', one_year_ago, today, api_key = av_key)
    
    df_dict[symbol] = stock
    time.sleep(30)

In [8]:
# Saving dictionary as pickle for later use 
pickle.dump( df_dict, open("stock_timeseries_dict.p", "wb" ) )

# To open pickle uncomment the following code:
# df_dict = pickle.load( open( "stock_timeseries_dict.p", "rb" ) )

In [9]:
df_dict = pickle.load( open("stock_timeseries_dict.p", "rb" ) )

### Accessing FRED API to scrape Germanys Risk Free Rate

In [10]:
# Access Long-Term Government Bond Yields of Germany via FRED API
URL_FRED = "https://api.stlouisfed.org/fred/series/observations?"
risk_params= {
    "api_key": "39ffc8e6f6c49a0a690ca8b875d15488",
    "series_id": "IRLTLT01DEM156N",
    "file_type": "json",
    "observation_start": "2020-01-01"
}

response_risk = requests.get(URL_FRED, params=risk_params)
json_risk = response_risk.json()["observations"]

df_risk = pd.DataFrame(json_risk)

In [11]:
df_risk.head()

,realtime_start,realtime_end,date,value
0,2022-05-30,2022-05-30,2020-01-01,-0.313636364
1,2022-05-30,2022-05-30,2020-02-01,-0.466
2,2022-05-30,2022-05-30,2020-03-01,-0.541363636
3,2022-05-30,2022-05-30,2020-04-01,-0.4495
4,2022-05-30,2022-05-30,2020-05-01,-0.5165


In [12]:
df_risk.dtypes

realtime_start    object
realtime_end      object
date              object
value             object
dtype: object

In [13]:
df_risk["value"]=df_risk["value"].astype("float")
df_risk["realtime_start"]=pd.to_datetime(df_risk["realtime_start"], format="%Y-%m-%d")
df_risk["realtime_end"]=pd.to_datetime(df_risk["realtime_end"], format="%Y-%m-%d")
df_risk["date"]=pd.to_datetime(df_risk["date"], format="%Y-%m-%d")

In [14]:
df_risk.dtypes

realtime_start    datetime64[ns]
realtime_end      datetime64[ns]
date              datetime64[ns]
value                    float64
dtype: object

In [15]:
#Transform percent value into decimal
df_risk["value"] =df_risk["value"].divide(100)

In [16]:
df_risk.head()

,realtime_start,realtime_end,date,value
0,2022-05-30,2022-05-30,2020-01-01,-0.003136
1,2022-05-30,2022-05-30,2020-02-01,-0.004660
2,2022-05-30,2022-05-30,2020-03-01,-0.005414
3,2022-05-30,2022-05-30,2020-04-01,-0.004495
4,2022-05-30,2022-05-30,2020-05-01,-0.005165


In [17]:
# Save data as csv and pickle
df_risk.to_csv("risk_free_rate.csv", index=False)
pickle.dump(df_risk, open("risk_free_rate.p", "wb" ))